In [2]:
import numpy as np
from collections import defaultdict
import pandas as pd
import treeswift as ts
import json
import re
import random
import string
from collections import Counter

In [58]:
with open("placement_tree.nwk", 'r') as f:
    tree_string = f.read()
tree_pp = ts.read_tree_newick(re.sub(r'{[0-9]+}', '', tree_string).strip())
ic = 0
node_set = list(tree_pp.traverse_postorder(leaves=True, internal=True))
avg_blen = {nd: tree_pp.extract_subtree(nd).avg_branch_length(internal=False) for nd in node_set}
for nd in node_set:
    nd.set_label(f"N{ic}")
    nd_p = nd.get_parent()
    if nd_p is not None:
        nd_n = ts.Node(f"P{ic}", nd.edge_length / 2)
        nd.set_edge_length(nd.edge_length / 2)
        nd_p.remove_child(nd)
        nd_p.add_child(nd_n)
        nd.set_parent(nd_n)
        nd_t = ts.Node(f"T{ic}", tree_pp.extract_subtree(nd).avg_branch_length(internal=False))
        # nd_t = ts.Node(f"T{ic}", 1e-7)
        nd_t.set_parent(nd_n)
        nd_n.add_child(nd_t)
        nd_n.add_child(nd)
    ic+=1

In [32]:
with open("all_samples.txt", 'r') as f:
    samples = Counter(map(lambda x: x.strip(), f.readlines()))
sample_profiles = defaultdict(lambda: defaultdict(float))
for s in samples:
    with open(f"hmi_placements/{s}.pplist", 'r') as f:
        pp_counts = Counter(map(lambda x: x.strip(), f.readlines()))
    # sample_profiles[s] = dict(map(lambda x: (f"T{x[0]}", x[1]/sum(pp_counts.values())), pp_counts.items()))
    sample_profiles[s] = dict(map(lambda x: (f"T{x[0]}", x[1]), pp_counts.items()))

KeyboardInterrupt: 

In [18]:
df = pd.DataFrame(sample_profiles)
df = df.fillna(0)

In [21]:
df.to_csv("./placement_table.tsv", sep="\t", index_label="#FeatureID")

In [59]:
tree_pp.write_tree_newick("./placement_tree_extended.nwk")